In [2]:
import json, os
from logging import warning
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# Annotation

In [45]:
for file in os.listdir("./annotation/"):
    print(file)

    drug_name = file.split(".")[0]
    
    with open("./annotation/" + file, "r") as f:

        data = f.readlines()


    nls = []

    for i in range(len(data)):
        # print(i)
        if(data[i] == '\n'):
            nls.append(i)

    prod_names = [i+1 for i in nls]
    prod_names = [0] + prod_names

    prod_names = [data[i] for i in prod_names]
    prod_names = [ele.split("\n")[0] for ele in prod_names]

    # prod_names    

    i = 0
    j = 1

    symptoms = {}
    cur_prod = prod_names[i]
    symp_list = []

    while(i<len(data) and j < len(data)):

        if(j in nls):
            symptoms[cur_prod] = symp_list

            i+= 1
            cur_prod = prod_names[i]
            symp_list = []
            j+= 2

        else:
            symp_list.append(data[j].split("\n")[0].strip())
            j+= 1

    symptoms[cur_prod] = symp_list

    with open("./drug_symptoms/"+drug_name+".json", "w") as f:
        json.dump(symptoms, f)

brufen.txt
paracetamol.txt
rantac.txt
pantoprazole.txt


In [3]:
counter = {}

for file in os.listdir("./drug_symptoms/"):

    with open("./drug_symptoms/" + file, "r") as f:
        data = json.load(f)

    for prod in data:
        for symp in data[prod]:
            if(symp in counter):
                counter[symp] += 1
            else:
                counter[symp] = 1

counter = {
    k : v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)
}

with open("symptoms_counter.json",'w') as f:
    json.dump(counter, f)

# Clustering 

In [46]:
with open("data_fever.json",'r') as f:
    data = json.load(f)

data

{'paracetamol': [{'product_name': 'Dolo 650 Tablet',
   'feedback': {'Patients taking Dolo 650 Tablet': [['Thrice A Day', '70%'],
     ['Once A Day', '20%'],
     ['Twice A Day', '10%']],
    'What are you using Dolo Tablet for?': [['Fever', '57%'],
     ['Pain relief', '22%'],
     ['Headache', '10%'],
     ['Others', '9%'],
     ['Pain during m*', '2%']],
    'How much was the improvement?': [['Average', '41%'],
     ['Excellent', '37%'],
     ['Poor', '22%']],
    'What were the side-effects while using Dolo 650 Tablet?': [['No Side Effec*',
      '89%'],
     ['Liver damage', '5%'],
     ['Allergic reac*', '3%']],
    'How do you take Dolo Tablet?': [['With food', '82%'],
     ['With or witho*', '13%'],
     ['Empty stomach', '6%']],
    'Please rate Dolo 650 Tablet on price': [['Not Expensive', '35%'],
     ['Average', '33%'],
     ['Expensive', '31%']]},
   'faq': [['What if I vomit after taking Dolo 650 Tablet?',
     'If you vomit in less than 30 minutes after having a dose of 

In [48]:
cols = ['Drug','Product','Query','Answer','Query_Type']
fever = pd.DataFrame(columns=cols)

fever.head()

,Drug,Product,Query,Answer,Query_Type


In [57]:
for drug in data:

    # print(drug + "\n\n\n")

    for product_details in data[drug]:
        
        # print(product_details)
        prod_name = product_details['product_name']

        # print(prod_name)

        for f in product_details['faq']:
            query = f[0]
            answer = f[1]

            fever = fever.append({'Drug':drug,'Product':prod_name,'Query':query,'Answer':answer,'Query_Type':'FAQ'},ignore_index=True)

        for p in product_details['patient_concerns']:
            query = p[0]
            answer = p[1]

            fever = fever.append({'Drug':drug,'Product':prod_name,'Query':query,'Answer':answer,'Query_Type':'Patient Concerns'},ignore_index=True)
    
        # for query in data[drug][product].keys():

            # for answer in data[drug][product][query]:

            #     fever = fever.append({
            #         'Drug': drug,
            #         'Product': product,
            #         'Query': query,
            #         'Answer': answer,
            #         'Query_Type': 'FAQ'
            #     }, ignore_index=True)

In [58]:
fever.shape

(2812, 5)

In [59]:
fever.head()

,Drug,Product,Query,Answer,Query_Type
0,paracetamol,Dolo 650 Tablet,What if I vomit after taking Dolo 650 Tablet?,If you vomit in less than 30 minutes after hav...,FAQ
1,paracetamol,Dolo 650 Tablet,When will I feel better after taking the Dolo ...,"Usually, you will start feeling better after a...",FAQ
2,paracetamol,Dolo 650 Tablet,How often can I take the Dolo 650 Tablet?,You should only take four doses of Dolo 650 Ta...,FAQ
3,paracetamol,Dolo 650 Tablet,Does Dolo 650 Tablet make babies sleepy?,"No, Dolo 650 Tablet does not make babies sleep...",FAQ
4,paracetamol,Dolo 650 Tablet,Is Dolo 650 Tablet safe for children?,Dolo 650 Tablet is considered safe for childre...,FAQ


In [63]:
fever.to_excel("fever.xlsx",index=False,engine = 'openpyxl')